## Enthalpy

> Given Zi = [0.26, 0.09, 0.25, 0.17, 0.11, 0.12], Temperature = 369.17 kelvin, Pressure = 21.7185 bar

In [2]:
# Constants for each component (C1, C2, C3, C4, C5)
vapor_pressure_constants = {
    'Methane': [39.205, -1324.4, -3.4366, 0.000031019, 2],
    'Ethane': [51.857, -2598.7, -5.1283, 0.000014913, 2],
    'Propane': [59.078, -3492.6, -6.0669, 0.000010919, 2],
    'Butane': [66.343, -4363.2, -7.046, 9.4509E-06, 2],
    'Pentane': [78.741, -5420.3, -8.8253, 9.6171E-06, 2],
    'Hexane': [104.65, -6995.5, -12.702, 0.000012381, 2],
#     'Propylene': [ 43.905, -3097.8 ,-3.4425, 1.00E-16, 6 ]
}

import numpy as np
np.set_printoptions(suppress=True, precision=6)

In [3]:

def bubbletemp(compositions,pressure):

    # Function to calculate K(T) for each component
    def calculate_K_b(T, component_constants, P):
        C1, C2, C3, C4, C5 = component_constants
        return np.exp(C1 + (C2 / T) + C3 * np.log(T) + (C4 * T**C5)) / P

    # Function to calculate the derivative of K(T) with respect to T
    def calculate_dK_dT(T, component_constants,P):
        C1, C2, C3, C4, C5 = component_constants
        exp_term = np.exp(C1 + (C2 / T) + C3 * np.log(T) + (C4 * T**C5)) / P
        dK_dT = exp_term * (
            -C2 / (T**2) +  # Derivative of C2 / T
            C3 / T +  # Derivative of C3 * log(T)
            C4 * C5 * T**(C5 - 1)  # Derivative of C4 * T^C5
        )
        return dK_dT

    # Function to calculate F(T)
    def F(T, constants, x, P):
        K_values = np.array([calculate_K_b(T, constants[comp], P) for comp in constants])
        return np.sum(K_values * x) - 1

    # Function to calculate F'(T)
    def F_prime(T, constants, x, P):
        K_values = np.array([calculate_K_b(T, constants[comp], P) for comp in constants])
        dK_dT_values = np.array([calculate_dK_dT(T, constants[comp],P) for comp in constants])
        return np.sum(x * dK_dT_values)

    # Newton-Raphson method to find bubble point temperature
    def newton_raphson_method_b(constants, x, P, initial_guess, tolerance=1e-2, max_iterations=100):
        T = initial_guess
        for _ in range(max_iterations):
            F_T = F(T, constants, x, P)
            F_prime_T = F_prime(T, constants, x, P)
            T_new = T - F_T / F_prime_T
            if abs(T_new - T) < tolerance:
                return T_new
            T = T_new
        raise ValueError("Newton-Raphson method did not converge")

    # Function to calculate vapor phase composition Yi
    def calculate_Yi(T, constants, x, P):
        K_values = np.array([calculate_K_b(T, constants[comp], P) for comp in constants])
    #     print(K_values)
        numerator = K_values * x
        denominator = np.sum(numerator)
        return numerator / denominator

    # Example initial guess
      # Temperature in Kelvin

    # Find bubble point temperature
    def bubblet(compositions,pressure):
        Y=[]
        initial_guess = 300
        x=compositions
        P=pressure
        P=P*10**5
        bubble_point_temperature = newton_raphson_method_b(vapor_pressure_constants, x, P, initial_guess)
    #     print(f"Bubble Point Temperature: {bubble_point_temperature:.2f} K")
        Yi = calculate_Yi(bubble_point_temperature, vapor_pressure_constants, x, P)
        Y.append(np.round(Yi, 6))
    #     print("Vapor Phase Compositions (Yi):")
        sum_Y=0
        for component, yi in zip(vapor_pressure_constants.keys(), Yi):
    #         print(f"{component}: {yi:.6f}")
            sum_Y=sum_Y+yi
    #     print(sum_Y)
        return bubble_point_temperature,Y
    y,z=bubblet(compositions,pressure)
    return y,z

In [4]:

bubbletemp([0.26, 0.09, 0.25, 0.17, 0.11, 0.12],21.785)

(211.180123857622,
 [array([0.980771, 0.014466, 0.00439 , 0.000343, 0.000027, 0.000003])])

In [5]:
def dewtemp(compositions,pressure):
    def calculate_K_d(T, component_constants, P):
        C1, C2, C3, C4, C5 = component_constants
        return np.exp(C1 + (C2 / T) + C3 * np.log(T) + (C4 * T**C5)) / P

    # Function to calculate the derivative of K(T) with respect to T
    def calculate_dK_dT(T, component_constants,P):
        C1, C2, C3, C4, C5 = component_constants
        exp_term = np.exp(C1 + (C2 / T) + C3 * np.log(T) + (C4 * T**C5)) / P
        dK_dT = exp_term * (
            -C2 / (T**2) +  # Derivative of C2 / T
            C3 / T +  # Derivative of C3 * log(T)
            C4 * C5 * T**(C5 - 1)  # Derivative of C4 * T^C5
        )
        return dK_dT

    # Function to calculate F(T)
    def F(T, constants, Yi, P):
        K_values = np.array([calculate_K_d(T, constants[comp], P) for comp in constants])
        return np.sum(Yi / K_values) - 1

    # Function to calculate F'(T)
    def F_prime(T, constants, Yi, P):
        K_values = np.array([calculate_K_d(T, constants[comp], P) for comp in constants])
        dK_dT_values = np.array([calculate_dK_dT(T, constants[comp],P) for comp in constants])
        return -np.sum(Yi * dK_dT_values / K_values**2)

    # Newton-Raphson method to find dew point temperature
    def newton_raphson_method(constants, Yi, P, initial_guess, tolerance=1e-6, max_iterations=100):
        T = initial_guess
        for _ in range(max_iterations):
            F_T = F(T, constants, Yi, P)
            F_prime_T = F_prime(T, constants, Yi, P)
            T_new = T - F_T / F_prime_T
            if abs(T_new - T) < tolerance:
                return T_new
            T = T_new
        raise ValueError("Newton-Raphson method did not converge")

    # Function to calculate liquid phase composition Xi
    def calculate_Xi(T, constants, Yi, P):
        K_values = np.array([calculate_K_d(T, constants[comp], P) for comp in constants])
        denominator = np.sum(Yi / K_values)
        return (Yi / K_values) / denominator


    def dewt(compositions,pressure):
        X=[]
        Yi=compositions
        P=pressure
        P=P*10**5
#         print(P)

        initial_guess = 100  # Temperature in Kelvin

        # Find dew point temperature
        dew_point_temperature = newton_raphson_method(vapor_pressure_constants, Yi, P, initial_guess)
    #     print(f"Dew Point Temperature: {dew_point_temperature:.2f} K")

        # Calculate liquid phase compositions
        Xi = calculate_Xi(dew_point_temperature, vapor_pressure_constants, Yi, P)
        X.append(Xi)
    #     print("Liquid Phase Compositions (Xi):")
        sum_X=0
        for component, xi in zip(vapor_pressure_constants.keys(), Xi):
    #         print(f"{component}: {xi:.4f}")
            sum_X=sum_X+xi
    #     print(sum_X)
        return dew_point_temperature, X
    x,z=dewt(compositions,pressure)
    return x,z

In [6]:
dewtemp([0.26, 0.09, 0.25, 0.17, 0.11, 0.12],21.785)

(401.74867205975943,
 [array([0.000857, 0.007743, 0.077079, 0.143835, 0.223217, 0.547268])])

In [7]:
# From the above calculations bubble T = 211 kelvin and dew T = 401.60.so the feed is mixture of liquid and vapour
# Do flash calculations and find vapour and liquid fraction.

In [8]:
import numpy as np

# Constants for each component (C1, C2, C3, C4, C5)
vapor_pressure_constants = {
    'Methane': [39.205, -1324.4, -3.4366, 0.000031019, 2],
    'Ethane': [51.857, -2598.7, -5.1283, 0.000014913, 2],
    'Propane': [59.078, -3492.6, -6.0669, 0.000010919, 2],
    'Butane': [66.343, -4363.2, -7.046, 9.4509E-06, 2],
    'Pentane': [78.741, -5420.3, -8.8253, 9.6171E-06, 2],
    'Hexane': [104.65, -6995.5, -12.702, 0.000012381, 2],
#     'Propylene': [ 43.905, -3097.8 ,-3.4425, 1.00E-16, 6 ]
}

# Molar fractions of the components
# zi = np.array([0, 0, 0.11, 0, 0, 0,0.89])
zi = np.array([0.26,0.09,0.25,0.17,0.11,0.12])
# zi = np.array([0.430, 0.1491, 0.414, 0.0069, 0, 0])

# Flash conditions
P =  21.7185*10**5 # Pressure in Pa
T = 369.17 # Temperature in K

# Calculate K values (vapor-liquid equilibrium ratios)
def calculate_K(T, constants):
    C1, C2, C3, C4, C5 = constants
    return np.exp(C1 + (C2 / T) + C3 * np.log(T) + (C4 * T**C5)) / P

# Print K values for each component
def print_K_values(vapor_pressure_constants, T, P):
#     print(f"K values at {T} K and {P} Pa:")
    for component, constants in vapor_pressure_constants.items():
        K = calculate_K(T, constants)
#         print(f"{component}: K = {K:.4f}")

# Function to calculate F(V) and its derivative
def flash_equilibrium(V, zi, K_values):
    Xi = zi / (V * (K_values - 1) + 1)
    Yi = K_values * Xi
    F_V = np.sum(Yi) - np.sum(Xi)
    F_prime_V = -np.sum(zi * (K_values - 1)**2 / (V * (K_values - 1) + 1)**2)
    return F_V, F_prime_V

# Newton-Raphson method to solve for V
def solve_flash_calculation(zi, vapor_pressure_constants, T, P):
    K_values = np.array([calculate_K(T, constants) for constants in vapor_pressure_constants.values()])
    
    # Print K values
    print_K_values(vapor_pressure_constants, T, P)
    
    V = V_initial
    tolerance = 1e-6
    max_iterations = 100
    iteration = 0

    while iteration < max_iterations:
        F_V, F_prime_V = flash_equilibrium(V, zi, K_values)
        V_new = V - F_V / F_prime_V

        if np.abs(V_new - V) < tolerance:
            break
        
        V = V_new
        iteration += 1
    
    if iteration == max_iterations:
        print("Warning: Maximum iterations reached without convergence.")

    Xi = zi / (V * (K_values - 1) + 1)
    Yi = K_values * Xi
    
    return V, Xi, Yi

# Initial guess for vapor fraction
V_initial = 0.5

# Calculate the flash calculation
V, Xi, Yi = solve_flash_calculation(zi, vapor_pressure_constants, T, P)

print(f"\nVapor Fraction (V): {V:.4f}")
print(f"Liquid Compositions (Xi): {Xi}")
print(f"Vapor Compositions (Yi): {Yi}")


Vapor Fraction (V): 0.7379
Liquid Compositions (Xi): [0.002516 0.016608 0.149036 0.229033 0.246731 0.356075]
Vapor Compositions (Yi): [0.351466 0.116071 0.285866 0.149029 0.061429 0.036139]


In [9]:
# At feed condition vapour phase compositon and liquid phase composition
zv=[0.351466, 0.116071, 0.285866, 0.149029, 0.061429, 0.036139]
zx=[0.002516, 0.016608, 0.149036, 0.229033, 0.246731, 0.356075]
q=1-0.737

## Vapour Enthalpy
> Basis: Vapour phase mole fraction(flash calc), Temperature.

In [10]:
import numpy as np
from scipy.integrate import quad

# Constants for each component
components_data = [
    [0.33298, 0.79933, 2.08690, 0.41602, 991.96],  # Methane
    [0.44256, 0.84737, 0.87224, 0.67130, 2430.4],  # Ethane
    [0.59474, 1.26610, 0.84431, 0.86165, 2482.7],  # Propane
    [0.80154, 1.62420, 0.84149, 1.05750, 2476.1],  # Butane
    [0.88050, 3.01100, 1.65020, 1.89200, 747.6],   # Pentane
    [1.04400, 3.52300, 1.69460, 2.36900, 761.6]    # Hexane
]

# Heat of formation values (in kJ/mol) converted from J/mol
heat_of_formation = {
    "Methane": -74520,
    "Ethane": -83820,
    "Propane": -104680,
    "Butane": -125790,
    "Pentane": -146760,
    "Hexane": -166940
}

# Function to calculate ideal gas heat capacity (Cp^0)
def ideal_gas_heat_capacity(C1, C2, C3, C4, C5, T):
    term1 = C1
    term2 = C2 * ((C3 / T) / (np.sinh(C3 / T)))**2
    term3 = C4 * ((C5 / T) / (np.cosh(C5 / T)))**2
    Cp0 = term1 + term2 + term3
    return Cp0

# Function to integrate Cp over the range for a given component
def Cp_component_at_T(T, C1, C2, C3, C4, C5):
    return ideal_gas_heat_capacity(C1 * 1e5, C2 * 1e5, C3 * 1e3, C4 * 1e5, C5, T)

# Method to calculate the enthalpy of the mixture
def enthalpyv(Yi, T):
    T_min = 298.15  # Reference temperature
    T_max = T       # Maximum temperature for integration
    Tref = 298.15   # Reference temperature for enthalpy calculation

    # List to store component enthalpies
    enthalpies = []
    mixture_enthalpy = 0

    # Iterate through each component
    for i, (C1, C2, C3, C4, C5) in enumerate(components_data):
        # Integrate Cp over the temperature range
        Cp_integral, _ = quad(Cp_component_at_T, T_min, T_max, args=(C1, C2, C3, C4, C5))
        
        # Calculate the average Cp over the temperature range
        Cp_avg = Cp_integral / (T_max - T_min)
        
        # Convert Cp_avg to kJ/(kmol·K)
        Cp_avg_kJ_per_kmol_K = Cp_avg / 1000
        
        # Calculate enthalpy: Hv = heat_of_formation + Cp_avg * (T - Tref)
        component_names = list(heat_of_formation.keys())
        H_id = Cp_avg_kJ_per_kmol_K * (T - Tref)
        H_v = heat_of_formation[component_names[i]] + H_id
        
        # Multiply enthalpy by the mole fraction (Yi)
        weighted_Hv = Yi[i] * H_v
        
        # Add to the total enthalpy of the mixture
        mixture_enthalpy += weighted_Hv
        
        # Store the enthalpy value for each component
        enthalpies.append(weighted_Hv)

    # Return the total enthalpy of the mixture
    return mixture_enthalpy

# Example usage:
y = [0.35146631, 0.11607132, 0.2858656 , 0.14902947, 0.06142868 ,0.03613858]
T = 369.17

mixture_enthalpy = enthalpyv(y, T)
print(f"Total enthalpy of the vapour mixture = {mixture_enthalpy:.2f} kJ/mol")


Total enthalpy of the vapour mixture = -94499.68 kJ/mol


## Comparison with Aspen
>Enthply as per aspen 94412.7 and we got 94499.68, the vapour mole fraction of pentane and hexane are high compared to aspen.Since we are going to use same antonie constants at all points, the net value (Heat duty of condenser or reboiler) sholud be same.

## Liquid Enthalpy
> Basis: liquid phase mole fraction(flash cal),Temperature.

In [11]:
import numpy as np
from scipy.integrate import quad

# Constants for vapor phase components
components_data_vapor = [
    [0.33298, 0.79933, 2.08690, 0.41602, 991.96],  # Methane
    [0.44256, 0.84737, 0.87224, 0.67130, 2430.4],  # Ethane
    [0.59474, 1.26610, 0.84431, 0.86165, 2482.7],  # Propane
    [0.80154, 1.62420, 0.84149, 1.05750, 2476.1],  # Butane
    [0.88050, 3.01100, 1.65020, 1.89200, 747.6],   # Pentane
    [1.04400, 3.52300, 1.69460, 2.36900, 761.6]    # Hexane
]

# Heat of formation values (in kJ/mol) converted from J/mol
heat_of_formation = {
    "Methane": -74520,
    "Ethane": -83820,
    "Propane": -104680,
    "Butane": -125790,
    "Pentane": -146760,
    "Hexane": -166940
}

# Constants for liquid phase components
components_data_liquid = [
    [1.0194, 0.26087, -0.14694, 0.22154, 190.564],  # Methane
    [2.1091, 0.60646, -0.55492, 0.32799, 305.32],   # Ethane
    [2.9209, 0.78237, -0.77319, 0.39246, 369.83],   # Propane
    [3.6238, 0.8337, -0.82274, 0.39613, 425.12],    # Butane
    [4.5087, 0.95886, -0.92384, 0.39393, 469.7],    # Pentane
    [4.3848, 0.34057, 0.063282, -0.017037, 507.6]   # Hexane
]

# Function to calculate ideal gas heat capacity (Cp^0)
def ideal_gas_heat_capacity(C1, C2, C3, C4, C5, T):
    term1 = C1
    term2 = C2 * ((C3 / T) / (np.sinh(C3 / T)))**2
    term3 = C4 * ((C5 / T) / (np.cosh(C5 / T)))**2
    Cp0 = term1 + term2 + term3
    return Cp0

# Function to integrate Cp over the range for a given component
def Cp_component_at_T(T, C1, C2, C3, C4, C5):
    return ideal_gas_heat_capacity(C1 * 1e5, C2 * 1e5, C3 * 1e3, C4 * 1e5, C5, T)

# Function to calculate liquid enthalpy change (dh)
def calculate_dh(T, C1, C2, C3, C4, Tc):
    if T > Tc:
        return 0  # If T > Tc, dh will be zero
    t = T / Tc
    z = C2 + C3 * t + C4 * t**2
    dh = C1 * (1 - t)**z
    return dh / 1000  # Convert to kJ/kmol

# Method to calculate the liquid enthalpy of the mixture
def enthalpyl(xi, T):
    T_min = 298.15  # Reference temperature
    T_max = T       # Maximum temperature for integration
    Tref = 298.15   # Reference temperature for enthalpy calculation

    # First, calculate vapor phase enthalpies as Hv
    enthalpies_vapor = []

    
    for i, (C1, C2, C3, C4, C5) in enumerate(components_data_vapor):
        # Integrate Cp over the temperature range
        Cp_integral, _ = quad(Cp_component_at_T, T_min, T_max, args=(C1, C2, C3, C4, C5))
        
        # Calculate the average Cp over the temperature range
        Cp_avg = Cp_integral / (T_max - T_min)
        
        # Convert Cp_avg to kJ/(kmol·K)
        Cp_avg_kJ_per_kmol_K = Cp_avg / 1000
        
        # Calculate enthalpy: Hv = heat_of_formation + Cp_avg * (T - Tref)
        component_names = list(heat_of_formation.keys())
        H_id = Cp_avg_kJ_per_kmol_K * (T - Tref)
        H_v = heat_of_formation[component_names[i]] + H_id
        
        enthalpies_vapor.append(H_v)
#     print(enthalpies_vapor)

    # Now calculate the liquid phase enthalpy H_liquid = H_v - dh
    liquid_enthalpies = []
    
    for i, (C1, C2, C3, C4, Tc) in enumerate(components_data_liquid):
        dh = calculate_dh(T, C1 * 1e7, C2, C3, C4, Tc)  # Adjust C1 for units
        
        # Calculate total liquid enthalpy: H_liquid = H_v - dh
        H_liquid = enthalpies_vapor[i] - dh
        
        # Multiply by the mole fraction (xi) for weighted liquid enthalpy
        weighted_H_liquid = xi[i] * H_liquid
        
        # Store the weighted liquid enthalpy for the mixture
        liquid_enthalpies.append(weighted_H_liquid)

    # Return the total liquid enthalpy of the mixture
    return sum(liquid_enthalpies)

# Example usage:
# x = [0, 0, 0.051899, 0.665235, 0.187189, 0.095677]
# T = 422.53

# total_liquid_enthalpy = enthalpyl(compo, T)
# print(f" liquid enthalpy of the mixture = {total_liquid_enthalpy:.2f} kJ/kmol")

In [12]:
enthalpyl(zx,369.17)

-151517.88680560148

## Comparison with aspen
> The liquid value (-151518.04) is high compared to aspen (-150580) because the liquid phase mole fraction for methane is high and ethane is low compared to aspen. The vapour pressure data plays role.

## Feed Enthalpy

In [13]:
# vapour fraction and liquid fraction obtained from flash calc.

Hf= (1-q)*enthalpyv(zv,369.17)+q*enthalpyl(zx,369.17)
print(f"The enthalpy of feed Hf {Hf:.2f} KJ/Kmol.")
# print(Hf*100/3600)

The enthalpy of feed Hf -109495.50 KJ/Kmol.


## Top Enthalpy (Condenser)

> Assume top pressure and calculate dew point temperature

## Case : Partial Condenser Duty (KJ/sec).
> Assume condenser acts a ideal stage and temperature will be dew point of the mixture at given pressure.

> The Reflux and Distillate flow obtained from material balance.

> The distillate will be in vapour phase (material balance) and liquid (Reflux) composition calculated from dew point.

> The feed entering the column will be in vapour phase and composition calculated from material balance across condenser.

> The feed temperature will be dew point temperature of stage 1, calculated using above composition.

In [14]:
condensert=dewtemp([0.43685, 0.151217, 0.41193300000000005, 0, 0, 0],22.6)[0] # Condenser temperature
y=[0.43685, 0.151217, 0.41193300000000005, 0, 0, 0] # Distillate composition
x=dewtemp(y,22.6)[1][0] # Reflux composition

In [15]:
y

[0.43685, 0.151217, 0.41193300000000005, 0, 0, 0]

In [16]:
condensert=bubbletemp([0.436242, 0.150997, 0.410496, 0.00226, 0, 0],22.6)[0] # Condenser temperature
x=[0.436242, 0.150997, 0.410496, 0.00226, 0, 0] # Distillate composition
y=bubbletemp(x,22.6)[1][0] # Reflux composition

In [17]:
y

array([0.986438, 0.011043, 0.002517, 0.000001, 0.      , 0.      ])

In [18]:
D=59.6 # Given
R=1.077 # Given
L=R*D
V=L+D
z1= (x[0]*L+y[0]*D)/(V)
z2= (x[1]*L+y[1]*D)/(V)
z3= (x[2]*L+y[2]*D)/(V)
z4= (x[3]*L+y[3]*D)/(V)

In [19]:
V

123.7892

In [20]:
z=[z1, z2,z3,0,0,0]
# print(z)
stage1t=dewtemp(z,22.6)[0]

In [21]:
print(f"Enthalpy of vapour entering condenser {enthalpyv(z, stage1t)}.")
print(f"Enthalpy of vapour leaving condenser {enthalpyv(y, condensert)}.")
print(f"Enthalpy of liquid leaving condenser{enthalpyl(x, condensert)}")

Enthalpy of vapour entering condenser -82576.14844706185.
Enthalpy of vapour leaving condenser -78289.11627450117.
Enthalpy of liquid leaving condenser-104156.76971915625


In [22]:
Qc=(enthalpyv(y, condensert)*D - (enthalpyl(x, condensert)*L)+enthalpyv(z, stage1t)*V )/3600
print(f"Heat duty of condenser {Qc} KJ/sec.")

Heat duty of condenser -2278.4241562351203 KJ/sec.


In [23]:
123*(enthalpyv(y, 194)-enthalpyl(x, 194))/3600

884.0095903411603

In [24]:
# At 324 enthalpy of vapour inlet to condenser 
z=[0.15561066666666667, 0.10165033333333333, 0.7427389999999999,0,0,0]

In [25]:
import numpy as np
from scipy.integrate import quad


y = [0.0250, 0.0769, 0.88, 0.00024, 0, 0]
x=y
# y = [0.1448, 0.09976676923076924, 0.7554332307692309, 0, 0, 0]
# y = [0.43685, 0.151217, 0.41193300000000005, 0, 0, 0]

# Constants for each component
components_data_vapor = [
    [0.33298, 0.79933, 2.08690, 0.41602, 991.96],  # Methane
    [0.44256, 0.84737, 0.87224, 0.67130, 2430.4],  # Ethane
    [0.59474, 1.26610, 0.84431, 0.86165, 2482.7],  # Propane
    [0.80154, 1.62420, 0.84149, 1.05750, 2476.1],  # Butane
    [0.88050, 3.01100, 1.65020, 1.89200, 747.6],   # Pentane
    [1.04400, 3.52300, 1.69460, 2.36900, 761.6]    # Hexane
]

# Heat of formation values (in kJ/mol) converted from J/mol
heat_of_formation = {
    "Methane": -74520,
    "Ethane": -83820,
    "Propane": -104680,
    "Butane": -125790,
    "Pentane": -146760,
    "Hexane": -166940
}

# Define the temperature range
T_min = 298.15
T_max = 324
Tref = 298.15
T = 324

# Function to calculate ideal gas heat capacity (Cp^0)
def ideal_gas_heat_capacity(C1, C2, C3, C4, C5, T):
    term1 = C1
    term2 = C2 * ((C3 / T) / (np.sinh(C3 / T)))**2
    term3 = C4 * ((C5 / T) / (np.cosh(C5 / T)))**2
    Cp0 = term1 + term2 + term3
    return Cp0

# Function to integrate Cp over the range for a given component
def Cp_component_at_T(T, C1, C2, C3, C4, C5):
    return ideal_gas_heat_capacity(C1 * 1e5, C2 * 1e5, C3 * 1e3, C4 * 1e5, C5, T)

# Calculate the total enthalpy of the mixture
enthalpies = []
mixture_enthalpy = 0

for i, (C1, C2, C3, C4, C5) in enumerate(components_data_vapor):
    # Integrate Cp over the temperature range
    Cp_integral, _ = quad(Cp_component_at_T, T_min, T_max, args=(C1, C2, C3, C4, C5))
    
    # Calculate the average Cp over the temperature range
    Cp_avg = Cp_integral / (T_max - T_min)
    
    # Convert Cp_avg to kJ/(kmol·K)
    Cp_avg_kJ_per_kmol_K = Cp_avg / 1000
    
    # Calculate enthalpy: Hv = heat_of_formation + Cp_avg * (T - Tref)
    component_names = list(heat_of_formation.keys())
    H_id = Cp_avg_kJ_per_kmol_K * (T - Tref)
    H_v = heat_of_formation[component_names[i]] + H_id
    enthalpies.append(H_v)

    weighted_Hv = y[i] * H_v
    
    # Add to the total enthalpy of the mixture
    mixture_enthalpy += weighted_Hv
    
    # Store the enthalpy value for each component
    

# Print the total enthalpy of the mixture
print(f"Total vapor enthalpy of the mixture = {mixture_enthalpy:.2f} kJ/mol")


# Constants for liquid enthalpy calculations
components_data_liquid = [
    [1.0194, 0.26087, -0.14694, 0.22154, 190.564],  # Methane
    [2.1091, 0.60646, -0.55492, 0.32799, 305.32],   # Ethane
    [2.9209, 0.78237, -0.77319, 0.39246, 369.83],   # Propane
    [3.6238, 0.8337, -0.82274, 0.39613, 425.12],    # Butane
    [4.5087, 0.95886, -0.92384, 0.39393, 469.7],    # Pentane
    [4.3848, 0.34057, 0.063282, -0.017037, 507.6]   # Hexane
]

# Function to calculate enthalpy change (dh) for a given component
def calculate_dh(T, C1, C2, C3, C4, Tc):
    if T > Tc:
        return 0  # If T > Tc, dh will be zero
    t = T / Tc
    z = C2 + C3 * t + C4 * t**2
    dh = C1 * (1 - t)**z
    dh=dh/1000 # KJ/Kmol
    return dh

# Calculate liquid enthalpy for each component and add to the total mixture enthalpy
liquid_enthalpies = []

for i, (C1, C2, C3, C4, Tc) in enumerate(components_data_liquid):
    dh = calculate_dh(T, C1 * 10**7, C2, C3, C4, Tc)  # Adjust C1 for units as needed
    
    # Convert dh to kJ/(kmol·K)
    dh_kJ_per_kmol_K = dh
  
    
    # Calculate total liquid enthalpy: H_liquid = H_v + dh
    H_liquid = enthalpies[i] - dh_kJ_per_kmol_K 
    
    # Multiply by the mole fraction (Yi) for weighted liquid enthalpy
    weighted_H_liquid = x[i] * H_liquid  

    liquid_enthalpies.append(weighted_H_liquid)
    
    bottom_mixture_enthalpy_liquid = sum(liquid_enthalpies)

# Print the total liquid enthalpy of the mixture
print(f"Total reboiler liquid enthalpy of the mixture = {bottom_mixture_enthalpy_liquid:.2f} kJ/Kmol")


Total vapor enthalpy of the mixture = -98596.50 kJ/mol
Total reboiler liquid enthalpy of the mixture = -109607.01 kJ/Kmol


In [26]:
(100717.39-88196.19)*60/3600

208.6866666666666

In [27]:
HD=88196.19*D/3600
hD=115987.52*L/3600

In [28]:
HD,hD

(1460.1369233333335, 2068.0961441066665)

In [29]:
96401.99*V/3600

3314.868116807778

In [30]:
6805.7127106944445-1464.8836588982656-6261.715142222223

-920.8860904260446

In [31]:
HD,hD

(1460.1369233333335, 2068.0961441066665)

In [32]:
270/1.79

150.83798882681563

In [33]:
7.719813888888889*2.25*59

1024.80529375

In [34]:
84233.16-84233.16

0.0

In [35]:
(117309.05-94743.44)*(3.25)*59/3600

1201.93214375

In [36]:
((94841.43-88196)/3600)*1

1.8459527777777758

In [37]:
import numpy as np

# Constants for each component (C1, C2, C3, C4, C5)
vapor_pressure_constants = {
    'Methane': [39.205, -1324.4, -3.4366, 0.000031019, 2],
    'Ethane': [51.857, -2598.7, -5.1283, 0.000014913, 2],
    'Propane': [59.078, -3492.6, -6.0669, 0.000010919, 2],
    'Butane': [66.343, -4363.2, -7.046, 9.4509E-06, 2],
    'Pentane': [78.741, -5420.3, -8.8253, 9.6171E-06, 2],
    'Hexane': [104.65, -6995.5, -12.702, 0.000012381, 2],
}

# Given vapor compositions (Yi) - assumed
yi = np.array([0.43685, 0.151217, 0.411933, 0, 0, 0])

# Flash conditions
P =  22.808 * 10**5  # Pressure in Pa
T = 311  # Temperature in K

# Known vapor fraction (V)
V = 0.7568  # Adjust this value based on your case

# Calculate K values (vapor-liquid equilibrium ratios)
def calculate_K(T, constants):
    C1, C2, C3, C4, C5 = constants
    return np.exp(C1 + (C2 / T) + C3 * np.log(T) + (C4 * T**C5)) / P

# Function to calculate the liquid composition (Xi) given Yi, V, and K values
def calculate_xi(yi, K_values, V):
    Xi = yi / K_values * (1 - V) / V
    return Xi

# Function to calculate feed composition (Zi)
def calculate_zi(yi, xi, V, L):
    zi = V * yi + L * xi
    return zi

# Calculate K values for each component
K_values = np.array([calculate_K(T, constants) for constants in vapor_pressure_constants.values()])

# Calculate the liquid composition (Xi) based on V and K-values
xi = calculate_xi(yi, K_values, V)

# Known liquid fraction (L = 1 - V)
L = 1 - V

# Calculate the feed composition (Zi) based on Yi and Xi
zi = calculate_zi(yi, xi, V, L)

print(f"Liquid Compositions (Xi): {xi}")
print(f"Feed Compositions (Zi): {zi}")


Liquid Compositions (Xi): [0.003908 0.020399 0.231496 0.       0.       0.      ]
Feed Compositions (Zi): [0.331559 0.119402 0.368051 0.       0.       0.      ]


In [38]:
import numpy as np

# Constants for each component (C1, C2, C3, C4, C5)
vapor_pressure_constants = {
    'Methane': [39.205, -1324.4, -3.4366, 0.000031019, 2],
    'Ethane': [51.857, -2598.7, -5.1283, 0.000014913, 2],
    'Propane': [59.078, -3492.6, -6.0669, 0.000010919, 2],
    'Butane': [66.343, -4363.2, -7.046, 9.4509E-06, 2],
    'Pentane': [78.741, -5420.3, -8.8253, 9.6171E-06, 2],
    'Hexane': [104.65, -6995.5, -12.702, 0.000012381, 2],
#     'Propylene': [ 43.905, -3097.8 ,-3.4425, 1.00E-16, 6 ]
}

# Molar fractions of the components
# zi = np.array([0, 0, 0.11, 0, 0, 0,0.89])
zi = np.array([0.33, 0.119, 0.36805062, 0.        , 0.       ,  0.        ])
# zi = np.array([0.430, 0.1491, 0.414, 0.0069, 0, 0])

# Flash conditions
P =  22.8006*10**5 # Pressure in Pa
T = 311 # Temperature in K

# Calculate K values (vapor-liquid equilibrium ratios)
def calculate_K(T, constants):
    C1, C2, C3, C4, C5 = constants
    return np.exp(C1 + (C2 / T) + C3 * np.log(T) + (C4 * T**C5)) / P

# Print K values for each component
def print_K_values(vapor_pressure_constants, T, P):
#     print(f"K values at {T} K and {P} Pa:")
    for component, constants in vapor_pressure_constants.items():
        K = calculate_K(T, constants)
#         print(f"{component}: K = {K:.4f}")

# Function to calculate F(V) and its derivative
def flash_equilibrium(V, zi, K_values):
    Xi = zi / (V * (K_values - 1) + 1)
    Yi = K_values * Xi
    F_V = np.sum(Yi) - np.sum(Xi)
    F_prime_V = -np.sum(zi * (K_values - 1)**2 / (V * (K_values - 1) + 1)**2)
    return F_V, F_prime_V

# Newton-Raphson method to solve for V
def solve_flash_calculation(zi, vapor_pressure_constants, T, P):
    K_values = np.array([calculate_K(T, constants) for constants in vapor_pressure_constants.values()])
    
    # Print K values
    print_K_values(vapor_pressure_constants, T, P)
    
    V = V_initial
    tolerance = 1e-6
    max_iterations = 100
    iteration = 0

    while iteration < max_iterations:
        F_V, F_prime_V = flash_equilibrium(V, zi, K_values)
        V_new = V - F_V / F_prime_V

        if np.abs(V_new - V) < tolerance:
            break
        
        V = V_new
        iteration += 1
    
    if iteration == max_iterations:
        print("Warning: Maximum iterations reached without convergence.")

    Xi = zi / (V * (K_values - 1) + 1)
    Yi = K_values * Xi
    
    return V, Xi, Yi

# Initial guess for vapor fraction
V_initial = 0.5

# Calculate the flash calculation
V, Xi, Yi = solve_flash_calculation(zi, vapor_pressure_constants, T, P)

print(f"\nVapor Fraction (V): {V:.4f}")
print(f"Liquid Compositions (Xi): {Xi}")
print(f"Vapor Compositions (Yi): {Yi}")


Vapor Fraction (V): 1.2124
Liquid Compositions (Xi): [ 0.007613  0.044458  0.76498  -0.       -0.       -0.      ]
Vapor Compositions (Yi): [ 0.273528  0.105943  0.43758  -0.       -0.       -0.      ]


In [39]:
import numpy as np
from scipy.integrate import quad

# Data for the hyperbolic liquid heat capacity formula
components_data_liquid_1 = [
    [65.708, 38883, -257.95, 614.07, 190.564],  # Methane
    [44.009, 89718, 918.77, -1886, 305.32],     # Ethane
    [62.983, 113630, 633.21, -873.46, 369.83]   # Propane
]

# Data for the polynomial liquid heat capacity formula
components_data_poly = [
    [191030, -1675, 12.5, -0.03874, 0.000046121, 425],   # Butane
    [159080, -270.5, 0.99537, 0, 0, 469],               # Pentane
    [172120, -183.78, 0.88734, 0, 0, 507]               # Hexane
]

# Data for ideal gas heat capacity above the critical temperature
components_data_gas = [
    [0.33298, 0.79933, 2.08690, 0.41602, 991.96,190.564],  # Methane
    [0.44256, 0.84737, 0.87224, 0.67130, 2430.4,305.32],  # Ethane
    [0.59474, 1.26610, 0.84431, 0.86165, 2482.7,369.83],  # Propane
    [0.80154, 1.62420, 0.84149, 1.05750, 2476.1,425],  # Butane
    [0.88050, 3.01100, 1.65020, 1.89200, 747.6,469],   # Pentane
    [1.04400, 3.52300, 1.69460, 2.36900, 761.6,507]    # Hexane
]

# Function for hyperbolic equation for liquid heat capacity
def liquid_heat_capacity(T, C1, C2, C3, C4, Tc):
    t = 1 - (T / Tc)
    Cp = (C1**2 / t) + C2 - (2 * C1 * C3 * t) - (C1 * C4 * t**2) - (C3**2 * (t**3 / 3)) - (C3 * C4 * (t**4 / 2)) - (C4**2 * (t**5 / 5))
    return Cp

# Function for polynomial equation for liquid heat capacity
def calculate_CpL(T, C1, C2, C3, C4, C5):
    CpL = C1 + C2 * T + C3 * T**2 + C4 * T**3 + C5 * T**4
    return CpL

# Function for ideal gas heat capacity (Cp^0) above the critical temperature
def ideal_gas_heat_capacity(C1, C2, C3, C4, C5, T):
    term1 = C1
    term2 = C2 * ((C3 / T) / (np.sinh(C3 / T)))**2
    term3 = C4 * ((C5 / T) / (np.cosh(C5 / T)))**2
    Cp0 = term1 + term2 + term3
    return Cp0

def Cp_component_at_T(T, C1, C2, C3, C4, C5):
    return ideal_gas_heat_capacity(C1 * 1e5, C2 * 1e5, C3 * 1e3, C4 * 1e5, C5, T)

# Define temperature limits
T_min = 298.15
T_max = 369.17

# Process components for hyperbolic liquid heat capacity
print("Results for components using the hyperbolic equation:")
for component in components_data_liquid_1:
    C1, C2, C3, C4, Tc = component
    
    if T_max <= Tc:
        # Integrate Cp over the temperature range
        Cp_integral, _ = quad(liquid_heat_capacity, T_min, T_max, args=(C1, C2, C3, C4, Tc))
        Cp_avg = Cp_integral / (T_max - T_min)
        Cp_avg_kJ_per_kmol_K = Cp_avg / 1000
        print(Cp_avg_kJ_per_kmol_K)

# Process components for polynomial liquid heat capacity
print("\nResults for components using the polynomial equation:")
for component in components_data_poly:
    C1, C2, C3, C4, C5, Tc = component
    
    if T_max <= Tc:
        # Integrate CpL over the temperature range
        CpL_integral, _ = quad(calculate_CpL, T_min, T_max, args=(C1, C2, C3, C4, C5))
        CpL_avg = CpL_integral / (T_max - T_min)
        CpL_avg_kJ_per_kmol_K = CpL_avg / 1000
        print(CpL_avg_kJ_per_kmol_K)

# Process components for ideal gas heat capacity when temperature > Tc
print("\nResults for components using ideal gas heat capacity (T > Tc):")
for component in components_data_gas:
    C1, C2, C3, C4, C5,Tc = component
    
    if T_max > Tc:
        # Integrate Cp over the temperature range
        Cp_gas_integral, _ = quad(Cp_component_at_T, T_min, T_max, args=(C1, C2, C3, C4, C5))
        Cp_gas_avg = Cp_gas_integral / (T_max - T_min)
        Cp_gas_avg_kJ_per_kmol_K = Cp_gas_avg / 1000
        print(Cp_gas_avg_kJ_per_kmol_K)


Results for components using the hyperbolic equation:
203.18512085886672

Results for components using the polynomial equation:
158.27358741957926
180.05688630195095
209.95960293408197

Results for components using ideal gas heat capacity (T > Tc):
37.218408363929164
56.880099594667044


In [40]:
import numpy as np

# Function to calculate enthalpy change (dh) for a given component
def calculate_dh(T, C1, C2, C3, C4, Tc):
    if T > Tc:
        return 0  # If T > Tc, dh will be zero
    t = T / Tc
    z = C2 + C3 * t + C4 * t**2
    dh = C1 * (1 - t)**z
    return dh

# Component data: [C1, C2, C3, C4, Tc]
components_data = [
    [1.0194, 0.26087, -0.14694, 0.22154, 190.564],  # Methane
    [2.1091, 0.60646, -0.55492, 0.32799, 305.32],   # Ethane
    [2.9209, 0.78237, -0.77319, 0.39246, 369.83],   # Propane
    [3.6238, 0.8337, -0.82274, 0.39613, 425.12],    # Butane
    [4.5087, 0.95886, -0.92384, 0.39393, 469.7],    # Pentane
    [4.3848, 0.34057, 0.063282, -0.017037, 507.6]   # Hexane
]

# Temperature in Kelvin
T = 369.17

# Calculate dh for each component and print the results
for i, (C1, C2, C3, C4, Tc) in enumerate(components_data):
    dh = calculate_dh(T, C1 * 10**7, C2, C3, C4, Tc)  # Adjust C1 for units as needed
    print(f"Component {i+1}: dh at {T} K = {dh / 1000:.4f} KJ/Kmol.K")
    



Component 1: dh at 369.17 K = 0.0000 KJ/Kmol.K
Component 2: dh at 369.17 K = 0.0000 KJ/Kmol.K
Component 3: dh at 369.17 K = 2299.7469 KJ/Kmol.K
Component 4: dh at 369.17 K = 15525.9974 KJ/Kmol.K
Component 5: dh at 369.17 K = 21641.6806 KJ/Kmol.K
Component 6: dh at 369.17 K = 26846.2272 KJ/Kmol.K


In [41]:
import numpy as np
from scipy.integrate import quad

# Mole fractions of each component (Yi)
y = [0.35146631, 0.11607132, 0.2858656 , 0.14902947, 0.06142868 ,0.03613858]
x = [0.00251649, 0.01660758, 0.14903604 ,0.22903338, 0.24673136 ,0.35607524]

# Constants for each component
components_data_vapor = [
    [0.33298, 0.79933, 2.08690, 0.41602, 991.96],  # Methane
    [0.44256, 0.84737, 0.87224, 0.67130, 2430.4],  # Ethane
    [0.59474, 1.26610, 0.84431, 0.86165, 2482.7],  # Propane
    [0.80154, 1.62420, 0.84149, 1.05750, 2476.1],  # Butane
    [0.88050, 3.01100, 1.65020, 1.89200, 747.6],   # Pentane
    [1.04400, 3.52300, 1.69460, 2.36900, 761.6]    # Hexane
]

# Heat of formation values (in kJ/mol) converted from J/mol
heat_of_formation = {
    "Methane": -74520,
    "Ethane": -83820,
    "Propane": -104680,
    "Butane": -125790,
    "Pentane": -146760,
    "Hexane": -166940
}

# Define the temperature range
T_min = 298.15
T_max = 369.17
Tref = 298.15
T = 369.17

# Function to calculate ideal gas heat capacity (Cp^0)
def ideal_gas_heat_capacity(C1, C2, C3, C4, C5, T):
    term1 = C1
    term2 = C2 * ((C3 / T) / (np.sinh(C3 / T)))**2
    term3 = C4 * ((C5 / T) / (np.cosh(C5 / T)))**2
    Cp0 = term1 + term2 + term3
    return Cp0

# Function to integrate Cp over the range for a given component
def Cp_component_at_T(T, C1, C2, C3, C4, C5):
    return ideal_gas_heat_capacity(C1 * 1e5, C2 * 1e5, C3 * 1e3, C4 * 1e5, C5, T)

# Calculate the total enthalpy of the mixture
enthalpies = []
mixture_enthalpy = 0

for i, (C1, C2, C3, C4, C5) in enumerate(components_data_vapor):
    # Integrate Cp over the temperature range
    Cp_integral, _ = quad(Cp_component_at_T, T_min, T_max, args=(C1, C2, C3, C4, C5))
    
    # Calculate the average Cp over the temperature range
    Cp_avg = Cp_integral / (T_max - T_min)
    
    # Convert Cp_avg to kJ/(kmol·K)
    Cp_avg_kJ_per_kmol_K = Cp_avg / 1000
    
    # Calculate enthalpy: Hv = heat_of_formation + Cp_avg * (T - Tref)
    component_names = list(heat_of_formation.keys())
    H_id = Cp_avg_kJ_per_kmol_K * (T - Tref)
    H_v = heat_of_formation[component_names[i]] + H_id
    enthalpies.append(H_v)
    # Multiply enthalpy by the mole fraction (Yi)
    weighted_Hv = y[i] * H_v
    
    # Add to the total enthalpy of the mixture
    mixture_enthalpy += weighted_Hv
    
    # Store the enthalpy value for each component
    

# Print the total enthalpy of the mixture
print(f"Total vapor enthalpy of the mixture = {mixture_enthalpy:.2f} kJ/mol")

# Constants for liquid enthalpy calculations
components_data_liquid = [
    [1.0194, 0.26087, -0.14694, 0.22154, 190.564],  # Methane
    [2.1091, 0.60646, -0.55492, 0.32799, 305.32],   # Ethane
    [2.9209, 0.78237, -0.77319, 0.39246, 369.83],   # Propane
    [3.6238, 0.8337, -0.82274, 0.39613, 425.12],    # Butane
    [4.5087, 0.95886, -0.92384, 0.39393, 469.7],    # Pentane
    [4.3848, 0.34057, 0.063282, -0.017037, 507.6]   # Hexane
]

# Function to calculate enthalpy change (dh) for a given component
def calculate_dh(T, C1, C2, C3, C4, Tc):
    if T > Tc:
        return 0  # If T > Tc, dh will be zero
    t = T / Tc
    z = C2 + C3 * t + C4 * t**2
    dh = C1 * (1 - t)**z
    return dh

# Calculate liquid enthalpy for each component and add to the total mixture enthalpy
liquid_enthalpies = []

for i, (C1, C2, C3, C4, Tc) in enumerate(components_data_liquid):
    dh = calculate_dh(T, C1 * 10**7, C2, C3, C4, Tc)  # Adjust C1 for units as needed
    
    # Convert dh to kJ/(kmol·K)
    dh_kJ_per_kmol_K = dh / 1000
    
    # Calculate total liquid enthalpy: H_liquid = H_v + dh
    H_liquid = enthalpies[i] + dh_kJ_per_kmol_K    
    # Multiply by the mole fraction (Yi) for weighted liquid enthalpy
    weighted_H_liquid = x[i] * H_liquid
    liquid_enthalpies.append(weighted_H_liquid)
    mixture_enthalpy_liquid = sum(liquid_enthalpies)

# Print the total liquid enthalpy of the mixture
print(f"Total liquid enthalpy of the mixture = {mixture_enthalpy_liquid:.2f} kJ/mol")


Total vapor enthalpy of the mixture = -94499.68 kJ/mol
Total liquid enthalpy of the mixture = -113922.69 kJ/mol
